# First thing first. Lets import all packages.

In [ ]:
import pandas as pd
import numpy as np
import random
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import grey
from bokeh.models import HoverTool
from bokeh.plotting import figure,output_file,show,save
import json

## Let's open the file and read it.

In [46]:
file = open('trending.json',encoding="utf8")
raw_data = json.load(file)
trending_videos_list = raw_data['collector']
df_tiktok_dataset = pd.DataFrame(trending_videos_list)
df_tiktok_dataset.head()

,id,text,createTime,authorMeta,musicMeta,covers,webVideoUrl,videoUrl,videoUrlNoWaterMark,videoMeta,diggCount,shareCount,playCount,commentCount,downloaded,mentions,hashtags
0,6907228749016714497,Confidence went 📈,1608214517,"{'id': '6825540583826768902', 'secUid': 'MS4wL...","{'musicId': '6790057285814061057', 'musicName'...",{'default': 'https://p16-sign-sg.tiktokcdn.com...,https://www.tiktok.com/@ninakleij/video/690722...,https://v77.tiktokcdn.com/ed1f811617d7b5e18b8d...,,"{'height': 1024, 'width': 576, 'duration': 15}",3710,50,44800,68,True,[],[]
1,6875468410612993286,Quiet Zone... follow me on insta: joeysofo. Co...,1600819763,"{'id': '6729292817489986566', 'secUid': 'MS4wL...","{'musicId': '6678927595137977094', 'musicName'...",{'default': 'https://p16-sign-sg.tiktokcdn.com...,https://www.tiktok.com/@joeysofo/video/6875468...,https://v77.tiktokcdn.com/ab935f1975cb8b69aebf...,,"{'height': 1024, 'width': 576, 'duration': 11}",55700,1817,838100,936,True,[@dwight_schnuute],[]
2,6898699405898059010,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,1606228625,"{'id': '6791901371429913601', 'secUid': 'MS4wL...","{'musicId': '6881287666340481026', 'musicName'...",{'default': 'https://p16-sign-sg.tiktokcdn.com...,https://www.tiktok.com/@jackeyephone/video/689...,https://v21.tiktokcdn.com/video/tos/alisg/tos-...,,"{'height': 960, 'width': 540, 'duration': 19}",936200,21100,15300000,27100,True,[],"[{'id': '23428', 'name': 'tiktok', 'title': ''..."
3,6902819837345533186,,1607187987,"{'id': '6574079584262094854', 'secUid': 'MS4wL...","{'musicId': '6881107586830076678', 'musicName'...",{'default': 'https://p16-sign-sg.tiktokcdn.com...,https://www.tiktok.com/@naomivaneeren/video/69...,https://v21.tiktokcdn.com/video/tos/alisg/tos-...,,"{'height': 1024, 'width': 576, 'duration': 13}",12900,197,94900,143,True,[],[]
4,6905635666588192002,小技です👟✨#tiktok教室#tutorial,1607843600,"{'id': '6586854399372394498', 'secUid': 'MS4wL...","{'musicId': '6649693417590754053', 'musicName'...",{'default': 'https://p16-sign-sg.tiktokcdn.com...,https://www.tiktok.com/@io.dreamer_mk/video/69...,https://v77.tiktokcdn.com/7622979a5ceb97c6da4e...,,"{'height': 1024, 'width': 576, 'duration': 22}",8805,198,115300,52,True,[],"[{'id': '1627704798586882', 'name': 'tiktok教室'..."


Each cell contains a dictionary. Let's get  more familar by opening the cell with info about the author of first video

In [48]:
a=df_tiktok_dataset.loc[0,'authorMeta']
a

{'id': '6825540583826768902',
 'secUid': 'MS4wLjABAAAA1FfFjRMUzr0hX2YPT7pRr7bCPQWa-kU_kVo3HLghB9xPQsjgNl0iIxCKKV7BO-4f',
 'name': 'ninakleij',
 'nickName': 'Nina',
 'verified': False,
 'signature': 'don’t tell my instagram I’m here\n19 👸🏼',
 'avatar': 'https://p16-sign-sg.tiktokcdn.com/aweme/1080x1080/tos-alisg-avt-0068/bf750c969d404f541623e9e7d697b873.jpeg?x-expires=1608649200&x-signature=vWwNAWq%2BLTZ0rSaqn34kKJblJ4A%3D'}

## Now lets 'open' those cells and add them in extra columns

In [49]:
# Let's expand the hashtag cell containing lists to multiple rows
df_tiktok_dataset = df_tiktok_dataset.explode('hashtags').explode('mentions')

In [51]:
def object_to_columns(dfRow, **kwargs):
    '''Function to expand cells containing dictionaries, to columns'''
    for column, prefix in kwargs.items():
        if isinstance(dfRow[column], dict):
            for key, value in dfRow[column].items():
                columnName = '{}.{}'.format(prefix, key)
                dfRow[columnName] = value
    return dfRow

# Expand certain cells containing dictionaries to columns
df_tiktok_dataset = df_tiktok_dataset.apply(object_to_columns, 
                            authorMeta='authorMeta',  
                            musicMeta='musicMeta',
                            covers='cover',
                            videoMeta='videoMeta',
                            hashtags='hashtag', axis = 1)

# Remove the original columns containing the dictionaries
df_tiktok_dataset = df_tiktok_dataset.drop(['authorMeta','musicMeta','covers','videoMeta','hashtags'], axis = 1)
df_tiktok_dataset

,authorMeta.avatar,authorMeta.id,authorMeta.name,authorMeta.nickName,authorMeta.secUid,authorMeta.signature,authorMeta.verified,commentCount,cover.default,cover.dynamic,...,musicMeta.playUrl,playCount,shareCount,text,videoMeta.duration,videoMeta.height,videoMeta.width,videoUrl,videoUrlNoWaterMark,webVideoUrl
0,https://p16-sign-sg.tiktokcdn.com/aweme/1080x1...,6825540583826768902,ninakleij,Nina,MS4wLjABAAAA1FfFjRMUzr0hX2YPT7pRr7bCPQWa-kU_kV...,don’t tell my instagram I’m here\n19 👸🏼,False,68,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,...,https://sf16-sg.tiktokcdn.com/obj/tos-alisg-v-...,44800,50,Confidence went 📈,15,1024,576,https://v77.tiktokcdn.com/ed1f811617d7b5e18b8d...,,https://www.tiktok.com/@ninakleij/video/690722...
1,https://p16-sign-va.tiktokcdn.com/musically-ma...,6729292817489986566,joeysofo,JoeySofo,MS4wLjABAAAAvkCSTiPWJm7Ctqp7AN3mauS_Bi8tVrbtBg...,Tinder couldn’t help me lose my virginity so I...,False,936,https://p16-sign-sg.tiktokcdn.com/obj/tos-mali...,https://p16-sign-sg.tiktokcdn.com/obj/tos-mali...,...,https://sf77-sg.tiktokcdn.com/obj/musically-ma...,838100,1817,Quiet Zone... follow me on insta: joeysofo. Co...,11,1024,576,https://v77.tiktokcdn.com/ab935f1975cb8b69aebf...,,https://www.tiktok.com/@joeysofo/video/6875468...
2,https://p16-sign-sg.tiktokcdn.com/aweme/1080x1...,6791901371429913601,jackeyephone,JackJacko,MS4wLjABAAAAsI8XQOceYtnIhIbLZLhvz24tOWdWYavlPe...,Zakelijk 📩 jackeyephone@gmail.com\nInstagram: ...,False,27100,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,...,https://sf16-sg.tiktokcdn.com/obj/tos-alisg-v-...,15300000,21100,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,19,960,540,https://v21.tiktokcdn.com/video/tos/alisg/tos-...,,https://www.tiktok.com/@jackeyephone/video/689...
2,https://p16-sign-sg.tiktokcdn.com/aweme/1080x1...,6791901371429913601,jackeyephone,JackJacko,MS4wLjABAAAAsI8XQOceYtnIhIbLZLhvz24tOWdWYavlPe...,Zakelijk 📩 jackeyephone@gmail.com\nInstagram: ...,False,27100,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,...,https://sf16-sg.tiktokcdn.com/obj/tos-alisg-v-...,15300000,21100,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,19,960,540,https://v21.tiktokcdn.com/video/tos/alisg/tos-...,,https://www.tiktok.com/@jackeyephone/video/689...
2,https://p16-sign-sg.tiktokcdn.com/aweme/1080x1...,6791901371429913601,jackeyephone,JackJacko,MS4wLjABAAAAsI8XQOceYtnIhIbLZLhvz24tOWdWYavlPe...,Zakelijk 📩 jackeyephone@gmail.com\nInstagram: ...,False,27100,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,...,https://sf16-sg.tiktokcdn.com/obj/tos-alisg-v-...,15300000,21100,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,19,960,540,https://v21.tiktokcdn.com/video/tos/alisg/tos-...,,https://www.tiktok.com/@jackeyephone/video/689...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,https://p77-sign-sg.tiktokcdn.com/musically-ma...,6798143079889470469,erinwilliams_1,Erin Williams,MS4wLjABAAAAdEeo7qMCWpjEHQIY8SBaOT4g79XSLKQz0l...,International Dressage Rider for GB 🇬🇧 \nIG: @...,False,874,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,...,https://sf77-sg.tiktokcdn.com/obj/tiktok-obj/7...,309000,235,"The collab you didn’t know you needed, myself ...",14,1022,576,https://v77.tiktokcdn.com/a6a5e4b4310213520be7...,,https://www.tiktok.com/@erinwilliams_1/video/6...
999,https://p77-sign-sg.tiktokcdn.com/musically-ma...,6798143079889470469,erinwilliams_1,Erin Williams,MS4wLjABAAAAdEeo7qMCWpjEHQIY8SBaOT4g79XSLKQz0l...,International Dressage Rider for GB 🇬🇧 \nIG: @...,False,874,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,...,https://sf77-sg.tiktokcdn.com/obj/tiktok-obj/7...,309000,235,"The collab you didn’t know you needed, myself ...",14,102